# Loading and saving models

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import os 

processor = WhisperProcessor.from_pretrained("bofenghuang/whisper-large-v2-cv11-german")
model = WhisperForConditionalGeneration.from_pretrained("/Users/svenasmussen/Documents/Code/huggingFaceModels/whisper-large-v2-cv11-german")
model.config.forced_decoder_ids = None

# Assuming `model` is your trained model and `processor` is your processor
save_directory = "models/whisper-large-v2-cv11-german"

# Create output directory if needed
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Save the model
model.save_pretrained(save_directory)

# Save the processor/tokenizer
processor.save_pretrained(save_directory)

# Transformation using pipe (only works for large models)

In [1]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
from transformers import pipeline
import librosa

processor = WhisperProcessor.from_pretrained("/Users/svenasmussen/Documents/Code/huggingFaceModels/whisper-large-v2-cv11-german")
model_sample_rate = processor.feature_extractor.sampling_rate

device = "cuda:0" if torch.cuda.is_available() else "cpu"

if torch.backends.mps.is_available():
    device = torch.device("mps")

pipe = pipeline(
    "automatic-speech-recognition",
    model="/Users/svenasmussen/Documents/Code/huggingFaceModels/whisper-large-v2-cv11-german",
    chunk_length_s=30,
    stride_length_s=10,
    device=device,
)

# Specify the path to your local audio file
audio_file = "AU-20231115-1359-0200.mp3"

prediction = pipe(audio_file, batch_size=1)

print(prediction['text'])

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.20s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 (Musik) Aber immer sind wir nicht ganz dicht. Alle nicht ganz dicht in der Kuroase. Und wie sind Sie heute da? Mir geht es gar nicht gut, Frau Dr. Pettmöller. Dieser Bahnstreik, ich wollte heute mal schön aufs Festland und dann ist plötzlich dieser Wesselsky wieder. Oh ja, wo kommt der nun wieder her, ne? Ja, dat is Wesselsky. Jahrelang sieht man ihn nicht, spürt man ihn nicht, denn plötzlich ploppt er wieder auf ans Tageslicht. Dat is wie Howard Carbondale. So ähnlich. Dat Phänomen Welselski tritt alle drei bis vier Jahre vier Jahre in Deutschland auf, permanent in der Glotze, dann Streik, aber dann auch wieder in der Versenkung für Jahre. Wie oft ist denn dieser Vulkan in Island immer? Das nervt ja auch nur. Jo, kann's nicht fliegen? Ja, aber Island ist nicht so oft wie Welselski oder Carpenthal. Gott! Der Edna, der kommt alle zwei Jahre ums Eck. Kannst mal sehen, ganz andere Taktik. Ja, von der Außergewöhnlichkeit der Erscheinung ist das seltenste der Polsprung soll auch demnächst 

# With manual setting of language:

In [1]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor, WhisperTokenizer, pipeline
import torch
import librosa

feature_extractor = WhisperFeatureExtractor.from_pretrained("/Users/svenasmussen/Documents/Code/huggingFaceModels/whisper-large-v2-cv11-german")
tokenizer = WhisperTokenizer.from_pretrained("/Users/svenasmussen/Documents/Code/huggingFaceModels/whisper-large-v2-cv11-german", language="german", task="transcribe")

model = WhisperForConditionalGeneration.from_pretrained("/Users/svenasmussen/Documents/Code/huggingFaceModels/whisper-large-v2-cv11-german")
forced_decoder_ids = tokenizer.get_decoder_prompt_ids(language="de", task="transcribe")

if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

asr_pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    feature_extractor=feature_extractor,
    tokenizer=tokenizer,
    chunk_length_s=30,
    stride_length_s=10,
    device=device,
)

/Users/svenasmussen/Documents/Code/audioTranscribe/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.08s/it]


In [2]:
from pydub import AudioSegment

# Specify the path to your local audio file
audio_file = "231106_1005_Habe-die-Ehre_Ludwig-Wallner-von-der-Gaststaette-Grossmar.mp3"

# Load audio file
audio = AudioSegment.from_mp3(audio_file)

# Slice the first minute
one_minute_audio = audio[:60000]  # pydub works in milliseconds

# Save to a temporary file
temp_file = "temp.mp3"
one_minute_audio.export(temp_file, format="mp3")

# Use the temporary file for prediction
prediction = asr_pipe(temp_file, batch_size=1, generate_kwargs={"forced_decoder_ids": forced_decoder_ids})

print(prediction['text'])

Be er Heimat, habe die Ehre! Grüß Sie ganz herzlich. Zehn Uhr ist es die Zeit, könnte man schon an ein Weißwurstfrühstück denken, ja, da haben Sie recht, das wäre jetzt genau das Richtige. Leider können wir Sie übers Radio nicht servieren, aber ein bisschen über die Weißwurst reden, das können wir natürlich schon. Denn wenn Sie in München fragen, wo gehe ich denn hin, wo kriege ich denn richtig gute Weißwurst, da würden viele Münchner sagen, ja, da musst zum Wallner in die Gaststätte Großmarkthalle in Sendling im Bauch von München. Und der Wirt, der Ludwig Wallner, der ist heute bei uns zu Gast. Schön, dass Sie sich Zeit genommen haben, da freuen wir uns. Grüß Gott miteinander, oder grüß Gott. Vielen Dank für die recht herzliche Einladung. Also wir haben jetzt hier bloß einen Kaffee stehen unter Wasser. Und und wie wär das jetzt die Uhrzeit? Sie fangen sehr früh an, haben Sie mir erzählt, kurz nach fünf ist es schon in Betrieb, der macht ja auch sieben auf. Hätten Sie da schon ein paar